In [1]:
import pandas as pd



In [157]:
import bisect

In [4]:
from PIL import Image
import numpy as np
import io

In [29]:
def load_parquet_dataset(filename):
    df = pd.read_parquet(filename)
    labels = np.asarray(df['label'])
    images = [ np.asarray(Image.open(io.BytesIO(x['bytes']))) for x in df['image']]
    scaled = np.asarray(images) / 255.0
    image_reshaped = scaled.reshape(*scaled.shape, 1)
    return image_reshaped, labels

In [13]:
scaled = np.asarray(images / 255.0)

In [10]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras

In [81]:
from keras import datasets, layers, models
from keras import regularizers

In [126]:
model = models.Sequential([
    layers.Conv2D(28, (3,3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(56, (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(56, (3,3), activation='relu')
])

model.add(layers.Flatten())
model.add(layers.Dense(56, activation='relu', kernel_regularizer=regularizers.L2(0.05)))
model.add(layers.Dense(10))




In [127]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [113]:
train_images, train_labels = load_parquet_dataset("train-00000-of-00001.parquet")

In [114]:
test_images, test_labels = load_parquet_dataset("test-00000-of-00001.parquet")

In [128]:
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.9378 - loss: 0.4269 - val_accuracy: 0.9765 - val_loss: 0.1346
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9737 - loss: 0.1461 - val_accuracy: 0.9827 - val_loss: 0.1141
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9786 - loss: 0.1243 - val_accuracy: 0.9858 - val_loss: 0.0974
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9816 - loss: 0.1085 - val_accuracy: 0.9836 - val_loss: 0.1040
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9831 - loss: 0.1022 - val_accuracy: 0.9839 - val_loss: 0.0961
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9851 - loss: 0.0928 - val_accuracy: 0.9865 - val_loss: 0.0919
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9852 - loss: 0.0894 - val_accuracy: 0.9892 - val_loss: 0.0740
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9866 -

In [85]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

313/313 - 1s - 3ms/step - accuracy: 0.9882 - loss: 0.0905
Test accuracy: 0.99


In [129]:
from keras import Model
extracted = Model(model.inputs, model.layers[-1].output)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [131]:
features[1]

array([ -3.3260803,  -1.5349885,  10.2836075,  -7.375522 ,  -1.6870499,
       -10.818754 ,  -3.4503078,  -4.160905 ,  -4.074495 , -10.990408 ],
      dtype=float32)

In [133]:
for i, pred in enumerate(features):
    j = np.argmax(pred)
    if test_labels[i] != j:
        print (pred, test_labels[i], j)

[-3.5806108 -0.2974934  0.1532602  1.6310124 -4.931209   1.8319684
 -4.056396  -1.360333  -5.872167  -3.6404903] 3 5
[  5.46947    -5.3369884  -6.571871  -11.335885   -2.1183667  -1.3117188
   4.6998115  -8.914141   -3.058738   -5.4189234] 6 0
[-4.5201426   1.0729369  -2.2723448  -5.128118    2.5140135  -4.271702
 -5.1161146   1.9245844  -2.502344   -0.27202854] 7 4
[ 5.9678736  -4.66742    -3.4009328  -8.178346    0.64446265 -2.7619712
  3.2896297  -8.11525    -1.7016737  -4.9017544 ] 6 0
[-7.988726  -1.5525353 -6.110363   2.919658  -6.6546316  6.017028
 -3.9612691 -2.588372  -8.0396595 -2.7883456] 3 5
[-2.1345859 -2.004668   2.6826618 -1.4371608 -1.6832762 -3.712546
 -3.0278234 -0.3957885  2.2886531 -3.8272307] 8 2
[-4.066554  -1.2419174  1.9176164 -7.5589066  2.1492543 -5.1958003
  2.2418826 -4.056139  -5.0897884 -6.756756 ] 2 6
[-3.909569    1.9195501   1.8856726   0.29856881 -4.0277367  -3.58332
 -4.945414    0.96078557 -2.614361   -3.04963   ] 2 1
[-5.988936  -0.502747  -2.62977 

In [161]:
def normalized_predictions(images, labels):
    assert(len(images) == len(labels))
    predictions = model.predict(images)
    normalized = np.array([x / np.sqrt(np.dot(x,x)) for x in predictions])
    return normalized
    

In [196]:
norms_train = normalized_predictions(train_images, train_labels)
norms_test = normalized_predictions(test_images, test_labels)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [198]:
norms_test[0]

array([-0.37355432, -0.02566617, -0.2309672 , -0.14557321, -0.04970531,
       -0.30704433, -0.55195504,  0.55261594, -0.2789246 ,  0.02601011],
      dtype=float32)

In [202]:
def fimi_encode(normalized, labels, levels):
    discreditized = [ np.array([bisect.bisect_left(levels, x) for x in pred]) for pred in normalized]
    total = len(levels) * discreditized[0].shape[0]
    fimi_lines = []
    for i, item in enumerate(discreditized):
        fimi = ''
        idx = 0
        for attr in item:
            for j in range(0, attr):
                if idx == 0 and j == 0:
                    fimi += str(j+idx)
                else:
                    fimi += " %d" % (j+idx)
            idx += len(levels)
        fimi += " | " + str(labels[i])
        fimi_lines.append(fimi)
    return fimi_lines


In [190]:
levels = [-0.5, 0.0, 0.25, 0.5, 0.75]

In [204]:
train_fimi = fimi_encode(norms_train, train_labels, levels)

In [205]:
test_fimi = fimi_encode(norms_test, test_labels, levels)

In [206]:
with open("train.fimi", "w") as f:
    f.writelines(train_fimi)

In [208]:
with open("test.fimi", "w") as f:
    f.writelines(test_fimi)